In [ ]:
# import matplotlib.pyplot as plthj
# import numpy as np
# import pandas as pd
# import os
# import torch
# import time
# from datetime import datetime

# def plot_training_metrics(
#     train_losses, 
#     val_losses, 
#     val_accuracies=None, 
#     learning_rates=None, 
#     save_dir='plots'
# ):
#     """
#     Plot training metrics including losses, accuracy, and learning rate.
    
#     Args:
#         train_losses: List of training losses per epoch
#         val_losses: List of validation losses per epoch
#         val_accuracies: List of validation accuracies per epoch (optional)
#         learning_rates: List of learning rates per epoch (optional)
#         save_dir: Directory to save the plots
#     """
#     # Create directory if it doesn't exist
#     os.makedirs(save_dir, exist_ok=True)
    
#     # Generate timestamp for unique filenames
#     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
#     # Set style for plots
#     plt.style.use('ggplot')
    
#     # Plot losses
#     plt.figure(figsize=(10, 6))
#     epochs = range(1, len(train_losses) + 1)
    
#     plt.plot(epochs, train_losses, 'b-o', label='Training Loss')
#     plt.plot(epochs, val_losses, 'r-o', label='Validation Loss')
    
#     plt.title('CLIP Training and Validation Loss')
#     plt.xlabel('Epochs')
#     plt.ylabel('Loss')
#     plt.legend()
#     plt.grid(True)
    
#     # Save the loss plot
#     loss_plot_path = os.path.join(save_dir, f'clip_losses_{timestamp}.png')
#     plt.savefig(loss_plot_path, dpi=300, bbox_inches='tight')
#     print(f"Loss plot saved to {loss_plot_path}")
#     plt.close()
    
#     # Plot validation accuracy if provided
#     if val_accuracies:
#         plt.figure(figsize=(10, 6))
#         plt.plot(epochs, val_accuracies, 'g-o')
#         plt.title('CLIP Validation Accuracy')
#         plt.xlabel('Epochs')
#         plt.ylabel('Accuracy (%)')
#         plt.grid(True)
        
#         # Save the accuracy plot
#         acc_plot_path = os.path.join(save_dir, f'clip_accuracy_{timestamp}.png')
#         plt.savefig(acc_plot_path, dpi=300, bbox_inches='tight')
#         print(f"Accuracy plot saved to {acc_plot_path}")
#         plt.close()
    
#     # Plot learning rate if provided
#     if learning_rates:
#         plt.figure(figsize=(10, 6))
#         plt.plot(epochs, learning_rates, 'purple', marker='o')
#         plt.title('Learning Rate Schedule')
#         plt.xlabel('Epochs')
#         plt.ylabel('Learning Rate')
#         plt.yscale('log')  # Log scale often better for LR visualization
#         plt.grid(True)
        
#         # Save the learning rate plot
#         lr_plot_path = os.path.join(save_dir, f'clip_lr_schedule_{timestamp}.png')
#         plt.savefig(lr_plot_path, dpi=300, bbox_inches='tight')
#         print(f"Learning rate plot saved to {lr_plot_path}")
#         plt.close()
    
#     # Combined plot with dual y-axis
#     if val_accuracies:
#         fig, ax1 = plt.subplots(figsize=(12, 7))
        
#         # Loss curves on left y-axis
#         ax1.set_xlabel('Epochs')
#         ax1.set_ylabel('Loss', color='tab:blue')
#         ax1.plot(epochs, train_losses, 'b-o', label='Training Loss')
#         ax1.plot(epochs, val_losses, 'r-o', label='Validation Loss')
#         ax1.tick_params(axis='y', labelcolor='tab:blue')
        
#         # Accuracy curve on right y-axis
#         ax2 = ax1.twinx()
#         ax2.set_ylabel('Accuracy (%)', color='tab:green')
#         ax2.plot(epochs, val_accuracies, 'g-o', label='Validation Accuracy')
#         ax2.tick_params(axis='y', labelcolor='tab:green')
        
#         # Add legend
#         lines1, labels1 = ax1.get_legend_handles_labels()
#         lines2, labels2 = ax2.get_legend_handles_labels()
#         ax1.legend(lines1 + lines2, labels1 + labels2, loc='center right')
        
#         plt.title('CLIP Training Metrics')
#         plt.grid(True, alpha=0.3)
        
#         # Save the combined plot
#         combined_plot_path = os.path.join(save_dir, f'clip_combined_metrics_{timestamp}.png')
#         plt.savefig(combined_plot_path, dpi=300, bbox_inches='tight')
#         print(f"Combined metrics plot saved to {combined_plot_path}")
#         plt.close()


# # Updated training function to track and plot metrics
# def train_clip_model_with_plots(
#     model,
#     train_loader,
#     val_loader,
#     num_epochs=10,
#     learning_rate=2e-5,
#     weight_decay=0.01,
#     scheduler_type="cosine",
#     warmup_epochs=1,
#     checkpoint_dir="checkpoints",
#     plots_dir="plots",
#     device=None
# ):
#     if device is None:
#         device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
#     # Move model to device
#     model = model.to(device)
    
#     # Create optimizer
#     optimizer = torch.optim.AdamW(
#         model.parameters(),
#         lr=learning_rate,
#         weight_decay=weight_decay
#     )
    
#     # Create scheduler based on type
#     scheduler = None
#     if scheduler_type == "cosine":
#         # Cosine decay from initial lr to near-zero
#         scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
#             optimizer, 
#             T_max=num_epochs - warmup_epochs,
#             eta_min=1e-6
#         )
        
#         # Optional: Warmup scheduler for first few epochs
#         if warmup_epochs > 0:
#             warmup_scheduler = torch.optim.lr_scheduler.LinearLR(
#                 optimizer, 
#                 start_factor=0.1, 
#                 end_factor=1.0, 
#                 total_iters=warmup_epochs
#             )
            
#     elif scheduler_type == "linear":
#         # Linear decay
#         scheduler = torch.optim.lr_scheduler.LinearLR(
#             optimizer,
#             start_factor=1.0,
#             end_factor=0.1,
#             total_iters=num_epochs
#         )
        
#     elif scheduler_type == "plateau":
#         # Reduce LR when validation loss plateaus
#         scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#             optimizer,
#             mode='min',
#             factor=0.5,
#             patience=2,
#             verbose=True
#         )
    
#     # Create directories if they don't exist
#     os.makedirs(checkpoint_dir, exist_ok=True)
#     os.makedirs(plots_dir, exist_ok=True)
    
#     # Metrics tracking
#     train_losses = []
#     val_losses = []
#     val_accuracies = []
#     learning_rates = []
    
#     best_val_acc = 0
#     best_model_path = None
    
#     # Training loop
#     for epoch in range(num_epochs):
#         current_lr = optimizer.param_groups[0]['lr']
#         learning_rates.append(current_lr)
        
#         print(f"\n== Epoch {epoch+1}/{num_epochs} ==")
#         print(f"Learning rate: {current_lr:.2e}")
        
#         # Training phase
#         train_loss, train_time = do_train(
#             model,
#             train_loader,
#             optimizer,
#             epoch,
#             device,
#             scheduler=(warmup_scheduler if epoch < warmup_epochs and warmup_epochs > 0 else None),
#             max_grad_norm=1.0
#         )
        
#         # Evaluation phase
#         val_loss, val_acc, val_time = do_eval(model, val_loader, device)
        
#         # Record metrics
#         train_losses.append(train_loss)
#         val_losses.append(val_loss)
#         val_accuracies.append(val_acc * 100)  # Convert to percentage
        
#         # Step the scheduler
#         if scheduler_type == "plateau":
#             scheduler.step(val_loss)
#         elif scheduler is not None and epoch >= warmup_epochs:
#             scheduler.step()
#         elif warmup_epochs > 0 and epoch < warmup_epochs:
#             warmup_scheduler.step()
        
#         # Save checkpoint if it's the best model so far
#         if val_acc > best_val_acc:
#             best_val_acc = val_acc
#             checkpoint_path = os.path.join(checkpoint_dir, f"clip_best_model_epoch{epoch+1}.pt")
            
#             # Save model state
#             torch.save({
#                 'epoch': epoch + 1,
#                 'model_state_dict': model.state_dict(),
#                 'optimizer_state_dict': optimizer.state_dict(),
#                 'val_acc': val_acc,
#                 'val_loss': val_loss,
#                 'train_losses': train_losses,
#                 'val_losses': val_losses,
#                 'val_accuracies': val_accuracies,
#                 'learning_rates': learning_rates
#             }, checkpoint_path)
            
#             best_model_path = checkpoint_path
#             print(f"🔥 New best model saved to {checkpoint_path}")
        
#         # Plot metrics every few epochs or at the end
#         if (epoch + 1) % 5 == 0 or epoch == num_epochs - 1:
#             plot_training_metrics(
#                 train_losses, 
#                 val_losses, 
#                 val_accuracies, 
#                 learning_rates,
#                 save_dir=plots_dir
#             )
    
#     # Final plots
#     plot_training_metrics(
#         train_losses, 
#         val_losses, 
#         val_accuracies, 
#         learning_rates,
#         save_dir=plots_dir
#     )
    
#     # Save training history to CSV
#     history_df = pd.DataFrame({
#         'epoch': list(range(1, num_epochs + 1)),
#         'train_loss': train_losses,
#         'val_loss': val_losses,
#         'val_accuracy': val_accuracies,
#         'learning_rate': learning_rates
#     })
    
#     history_path = os.path.join(plots_dir, f'training_history_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv')
#     history_df.to_csv(history_path, index=False)
#     print(f"Training history saved to {history_path}")
    
#     print(f"\n== Training Complete ==")
#     print(f"Best validation accuracy: {best_val_acc*100:.2f}%")
#     print(f"Best model saved at: {best_model_path}")
    
#     return model, best_model_path, history_df


# # Example usage with the do_train and do_eval functions from previous code
# def do_train(model, train_loader, optimizer, epoch, device, scheduler=None, max_grad_norm=1.0):
#     model.train()
#     total_loss = 0.0
#     start_time = time.time()
    
#     for batch_idx, batch in enumerate(train_loader):
#         batch = {k: v.to(device) for k, v in batch.items()}
        
#         # Forward pass
#         outputs = model(**batch, return_loss=True)
#         loss = outputs.loss
        
#         # Backward pass
#         loss.backward()
        
#         # Gradient clipping for stability
#         if max_grad_norm > 0:
#             torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            
#         optimizer.step()
#         optimizer.zero_grad()
        
#         total_loss += loss.item()
        
#         # Report progress for long epochs
#         if batch_idx % 50 == 0:
#             print(f"  Batch {batch_idx}/{len(train_loader)} - Loss: {loss.item():.4f}")
    
#     train_time = time.time() - start_time
#     avg_train_loss = total_loss / len(train_loader)
    
#     print(f"✅ Epoch {epoch+1} - Train Loss: {avg_train_loss:.4f} - Time: {train_time:.2f}s")
#     return avg_train_loss, train_time

# @torch.no_grad()
# def do_eval(model, val_loader, device):
#     model.eval()
#     total_loss = 0.0
#     all_preds = []
#     all_labels = []
#     start_time = time.time()
    
#     for batch in val_loader:
#         batch = {k: v.to(device) for k, v in batch.items()}
        
#         # Get loss
#         outputs = model(**batch, return_loss=True)
#         loss = outputs.loss
#         total_loss += loss.item()
        
#         # Get image and text features
#         image_embeds = model.get_image_features(pixel_values=batch['pixel_values'])
#         text_embeds = model.get_text_features(
#             input_ids=batch['input_ids'], 
#             attention_mask=batch['attention_mask']
#         )
        
#         # Normalize
#         image_embeds = image_embeds / image_embeds.norm(dim=-1, keepdim=True)
#         text_embeds = text_embeds / text_embeds.norm(dim=-1, keepdim=True)
        
#         # Compute similarity
#         logits = torch.matmul(image_embeds, text_embeds.T) * model.logit_scale.exp()
        
#         # Prediction (diagonal elements should be highest)
#         preds = torch.argmax(logits, dim=1)
#         labels = torch.arange(len(preds)).to(device)
        
#         all_preds.extend(preds.cpu().tolist())
#         all_labels.extend(labels.cpu().tolist())
    
#     val_time = time.time() - start_time
#     avg_val_loss = total_loss / len(val_loader)
#     val_acc = len([i for i, (p, l) in enumerate(zip(all_preds, all_labels)) if p == l]) / len(all_preds)
    
#     print(f"📊 Validation Loss: {avg_val_loss:.4f} - Val_Accuracy: {val_acc*100:.2f}% - Time: {val_time:.2f}s")
#     return avg_val_loss, val_acc, val_time


# # Example of how to use the plotting functionality
# if __name__ == "__main__":
#     from transformers import CLIPModel, CLIPProcessor
    
#     # Load pre-trained CLIP model and processor
#     model_name = "openai/clip-vit-base-patch32"
#     model = CLIPModel.from_pretrained(model_name)
#     processor = CLIPProcessor.from_pretrained(model_name)
    
#     # Prepare your datasets and dataloaders here
#     # train_dataset = ...
#     # val_dataset = ...
#     # train_loader = ...
#     # val_loader = ...
    
#     # Train the model with plotting
#     trained_model, best_model_path, history_df = train_clip_model_with_plots(
#         model=model,
#         train_loader=train_loader,
#         val_loader=val_loader,
#         num_epochs=20,
#         learning_rate=2e-5,
#         scheduler_type="cosine",
#         warmup_epochs=2,
#         plots_dir="clip_training_plots"
#     )
    
#     # If you want to plot metrics from a saved checkpoint:
#     """
#     checkpoint = torch.load('path/to/checkpoint.pt')
#     plot_training_metrics(
#         checkpoint['train_losses'],
#         checkpoint['val_losses'],
#         checkpoint['val_accuracies'],
#         checkpoint['learning_rates'],
#         save_dir='restored_plots'
#     )
#     """